In [42]:

from konlpy.tag import Kkma #pip install jpype1 #pip install konlpy

from konlpy.tag import Twitter

from konlpy.tag import Mecab

from collections import Counter #워드 클라우드에서 단어들을 쉽게 집계하기 위해 사용

from sklearn.feature_extraction.text import TfidfVectorizer #pip install scikit-learn

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import normalize

from ole import OleFile #pip install ole-py

from wordcloud import WordCloud


from gensim.models import word2vec
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
#from gensim.models.doc2vec import Doc2Vec
import multiprocessing


wikimodel_dir='C:/doc2vec/docmodel/w2v.model'
wikimodel_dir2='C:/doc2vec/model/wiki.model'

dgu_dir='C:/doc2vec/test/Dgu1.txt'
query_dir='C:/doc2vec/query/query.txt'


model = Word2Vec.load(wikimodel_dir2)

import numpy as np

import io
import re
import os
import pytagcloud
import simplejson
import sys

import openpyxl # excel파일 읽기

import olefile # 워드파일읽기

import matplotlib.pyplot as plt #워드클라우드

from google.cloud import vision
from google.cloud.vision import types

from pdfminer.pdfinterp import PDFResourceManager, process_pdf #pip install pdfminer3k
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from io import open #pdf 파일 읽기위함

stopwords = ['번호', '작성자', '작성', '대학', '회수','안내', '공지', '게시','내용','보기','보내기','사항', '학생', '기간', '첨부파일', '학기','신청','학점','학년','지원',
             '국제','방법','확인','선발','지원','모집','교류','실습', '첨부파일','다운로드', '결과']

stopwords2 = []


#i=0



In [236]:

dict_m=[]
labels= ['척추동물', '돼지', '수퇘지', '멧돼지', '주둥이', '페커리', '동물', '사파리', '가축']
print(labels)

class SentenceTokenizer(object):

    def __init__(self):

        self.twitter = Twitter()
        self.kkma = Kkma()
        
        
    def text2sentences(self, text):
        sentences22 = self.kkma.sentences(text)
        for idx in range(0, len(sentences22)):
            if len(sentences22[idx]) <= 10:
                sentences22[idx-1] += (' ' + sentences22[idx])
                sentences22[idx] = ''

        return sentences22





    def get_nouns(self, sentences):

        nouns = []

        for sentence in sentences:

            if sentence is not '':

                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))

                                       if noun not in stopwords and len(noun) > 1]))

        return nouns


class GraphMatrix(object):

    def __init__(self):
        self.tfidf = TfidfVectorizer()

        self.cnt_vec = CountVectorizer()

        self.graph_sentence = []

    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        
        #print('문장 가공전2222',tfidf_mat)
        #print('크기2222',tfidf_mat.shape)

        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)

        return self.graph_sentence
    
    #labels= ['척추동물', '돼지', '수퇘지', '멧돼지', '주둥이', '페커리', '동물', '사파리', '가축']
    
    def build_words_graph(self, sentence):
        
        cnt_vec_mat2=self.cnt_vec.fit_transform(sentence).toarray().astype(float)
        
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        
        vocab = self.cnt_vec.vocabulary_
        
        sorted_vocab_idx = sorted(vocab.items())
        
        print('sorted_vocab_idx', sorted_vocab_idx)
        
        print(len(sorted_vocab_idx))
        
        for d,c in sorted_vocab_idx :
            dict_m.append(d)
        
        for i in labels :
            cnt_col=0
            for j in dict_m:
                #print('j',j)
                cnt_col+=1
                model_sim = model.wv.similarity(i,j)
                cnt_vec_mat2[:,(cnt_col-1):cnt_col]+=model_sim

        cnt_vec_mat99 = normalize(cnt_vec_mat2.astype(float), axis=0)
        
        total_cnt=np.dot(cnt_vec_mat99.T, cnt_vec_mat99)
        
        total_origin=np.dot(cnt_vec_mat.T, cnt_vec_mat)
        
        print(total_cnt)
        
        print(total_origin)
        
        

        return total_origin, {vocab[word]: word for word in vocab}


class Rank(object):

    def get_ranks(self, graph, d=0.85):  # d = damping factor

        A = graph

        matrix_size = A.shape[0]

        for id in range(matrix_size):

            A[id, id] = 0  # diagonal 부분을 0으로

            link_sum = np.sum(A[:, id])  # A[:, id] = A[:][id]

            if link_sum != 0:
                A[:, id] /= link_sum

            A[:, id] *= -d

            A[id, id] = 1

        #print('A\n',A)
        B = (1 - d) * np.ones((matrix_size, 1))
        #print('B\n',B)

        ranks = np.linalg.solve(A, B)  # 연립방정식 Ax = b

        return {idx: r[0] for idx, r in enumerate(ranks)}


class TextRank(object):

    def __init__(self, text):

        self.sent_tokenize = SentenceTokenizer()
        
        #self.sentences = self.sent_tokenize.text2sentences(text)

        self.sentences=text
        #print('\n문장 개수',len(self.sentences))
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)

        #print('nouns\n',nouns)

        self.graph_matrix = GraphMatrix()

        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        
        #print(''self.sent_graph)

        #print('self.sent_graph\n',self.sent_graph)
        #print(self.sent_graph.shape)

        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

        print('words_graph\n',self.words_graph)
        print(self.words_graph.shape)

        self.rank = Rank()

        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        
        #print('self.sent_rank_idx',self.sent_rank_idx)
        
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        
        #print('self.sorted_sent_rank_idx\n\n',self.sorted_sent_rank_idx)

        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        
        print('self.word_rank_idx',self.word_rank_idx)

        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        
        print('self.sorted_word_rank_idx',self.sorted_word_rank_idx)

    def summarize(self, sent_num=5):

        summary = []

        index = []

        if len(self.sentences) > 30 :
            sent_num=7

        elif len(self.sentences) > 100:
            sent_num=10

        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)

        index.sort()

        for idx in index:
            summary.append(self.sentences[idx])

        return summary

    def keywords(self, word_num=10):

        rank = Rank()

        rank_idx = rank.get_ranks(self.words_graph)

        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []

        index = []

        #if len(self.sentences) > 30 :
            #word_num=15

        #elif len(self.sentences) > 100:
            #word_num=20

        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        # index.sort()

        for idx in index:
            keywords.append(self.idx2word[idx])

        return keywords

def read_pdf_file(pdfFile):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)

    process_pdf(rsrcmgr, device, pdfFile)
    device.close()

    content = retstr.getvalue()
    retstr.close()
    return content




    






['척추동물', '돼지', '수퇘지', '멧돼지', '주둥이', '페커리', '동물', '사파리', '가축']


In [237]:
path_dir='C:/doc2vec/word_pre/txt/'
path_dir2='C:/doc2vec/word_pre/txt2/'
path_dir3='C:/doc2vec/word_pre/txt3/'
path_dir4='C:/doc2vec/word_pre/txt4/'
path_dir5='C:/doc2vec/word_pre/txt5/'
path_dir6='C:/doc2vec/word_pre/txt6/'

jpg_dir='C:/doc2vec/word_pre/picture/'
pdf_dir='C:/doc2vec/word_pre/pdf/'
hwp_dir='C:/doc2vec/word_pre/hwp/'

path_pre='C:/doc2vec/result/pre_word/'
summary_dir='C:/doc2vec/word_result/summary_test/'
cloud_dir = 'C:/doc2vec/word_pre/cloud/'
keyword_before='C:/doc2vec/result/keyword_before/'
full_dir='C:/doc2vec/word_result/total_test/'
keyword_after='C:/doc2vec/result/keyword_after/'


txt_list=os.listdir(path_dir) #폴더안에 있는 파일 리스트로 저장
txt_list2=os.listdir(path_dir2)
txt_list3=os.listdir(path_dir3)
txt_list4=os.listdir(path_dir4)
txt_list5=os.listdir(path_dir5)
txt_list6=os.listdir(path_dir6)

txt_pre=os.listdir(path_pre)
str1=''
total_word=[]
total_word2=[]
t = Twitter()
#dir=path_dir
sys.stdout.flush()

for change in txt_pre:
    total = []  # 통합문서

    remover= change.replace(".txt","-")
    original=path_pre+change
    etc_pdf= pdf_dir+ remover + '0.pdf'
    etc_hwp = hwp_dir + remover + '0.hwp'
    jpg_name = change.replace("txt", "jpg")
    jpg_file=jpg_dir+jpg_name
    png_name = change.replace("txt", "png")
    png_file = jpg_dir + png_name
    pdf_name = change.replace("txt", "pdf")
    pdf_file = pdf_dir + pdf_name
    hwp_name = change.replace("txt", "hwp")
    hwp_file = hwp_dir + hwp_name
    textocr=""

    
    print('\n오리지날',original)

    input_file = open(original, "r", encoding="utf-8")
    text1 = input_file.readlines() #문서 불러오고 한줄씩 읽기
    total += text1
    input_file.close()
    
    if(os.path.isfile(jpg_file)==True ) :
        client = vision.ImageAnnotatorClient()
        file_name = os.path.join(
            os.path.dirname(__file__),
            jpg_file)

        with io.open(file_name, 'rb') as image_file:
            content = image_file.read()

        image = vision.types.Image(content=content)

        response = client.text_detection(image=image)
        texts = response.text_annotations
        textocr = texts[0].description

        jpg1 = ""
        jpg_list = []  # 이미지 리스트
        for x in textocr:
            if (x == '\n'):
                jpg_list.append(jpg1 + '\n')
                jpg1 = ""
            else:
                jpg1 = jpg1 + x
        print('jpg 있음')
        print()
        total += jpg_list

    #print('글 내용',total)



    #total = text1 + jpg_list
    #print('\n전체 내용',total)
    

    final = [] # 통합문 리스트
    for word in total:

        if (len(word) == 1 or len(word) == 2):
            pass

        else:
            final.append(word)
    # 불필요한 \n 없애기위함

    print('결과 total:\n', final)
    print(len(final))

    total_word=t.nouns(str(final))
    total_word2+=total_word


    #print('원래텍스트 타입', (type(text1)))
    #print('text1:', text1)

    full_name=full_dir+change
    full = open(full_name, 'w', encoding="utf-8")
    full.writelines(final)  # 리스트 형태인 sum2를 저장하기위해 씀

    full.close()


    textrank = TextRank(final)




    sum2=textrank.summarize() # 요약문 함수
    keyword_1=textrank.keywords()
    #newsum=str(sum2)

    temp=summary_dir+change

    f = open(temp, 'w',encoding="utf-8")
    f.writelines(sum2) #리스트 형태인 sum2를 저장하기위해 씀
    f.close()

    keyword_temp=keyword_after+change
    keyword_save = open(keyword_temp, 'w', encoding='utf-8')
    for i in keyword_1 :
        keyword_save.writelines(i)
        keyword_save.writelines(" ")

    #keyword_save.writelines(keyword_1)
    keyword_save.close()
    
    
    #파일 저장



오리지날 C:/doc2vec/result/pre_word/test10.txt
결과 total:
 ['\ufeff강원도 철원의 야생 멧돼지 폐사체 등에서 아프리카돼지열병(ASF) 2건이 추가로 확인됐다. \n', 'ASF 검출 사례는 2건이 추가되면서 전국에서 41건으로 늘었다.\n', '환경부 소속 국립환경과학원은 지난 4일 오후 10시 30분쯤 민통선 남쪽 3㎞ 지점인 철원군 서면에서 총기로 포획한 멧돼지와 5일 오전 11시쯤 민통선 남쪽 13㎞ 철원군 갈말읍에서 발견된 멧돼지 폐사체에서 ASF 바이러스가 각각 검출됐다고 7일 밝혔다. \n', '이에 따라 멧돼지에서 ASF바이러스가 검출된 사례는 전국에서 41건, 철원에서 15건으로 늘었다.\n', '야생 멧돼지가 발견된 철원 갈마읍에는 10㎞이내에 46개 농가(철원군 31, 포천 15개)가 있다. \n', '이들 농가에서는 돼지 약 10만5000마리를 사육 중이다. \n', '멧돼지를 총기로 포획한 철원군 서면은 발견지점에서 10㎞이내에 12개의 농가가 있다. \n', '이곳에선 약 4만1000마리의 돼지를 사육하고 있다.\n', '농식품부는 ASF 바이러스 검출이 확인된 직후 반경 10㎞ 내 농가와 경기, 강원 전체 양돈 농가에 문자메시지를 전송해 이 사실을 알렸다. \n', '또 농장 내부 소독과 울타리 방역 시설을 점검했다. \n', '철원군과 포천시에는 양성 개체 발견 지점 10㎞ 내 농가에 이동 제한 조치를 내리기도 했다. \n', '농식품부는 이날 철원군과 인접 지역인 화천군, 포천시에 직원들을 파견해 지역 내 농가 울타리를 점검하고 방역 조치가 제대로 이뤄졌는지 확인할 계획이다.']
12
sorted_vocab_idx [('각각', 0), ('갈말읍', 1), ('강원', 2), ('강원도', 3), ('개체', 4), ('경기', 5), ('계획', 6), ('국립', 7), ('남쪽', 8), ('내부', 9), ('농가', 10), ('농식품부', 11), ('농장', 12), ('돼지

C:\Users\kang\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\kang\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
